In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
from google.colab import drive
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Exemplo de aplicação de pré-processamento a uma imagem
def preprocess_image(imagem, resize=True, W=100, L=100):
    # Aqui, você pode realizar qualquer pré-processamento adicional necessário para a imagem
    imagens_preprocessadas = tf.cast(imagem, tf.float32)

    if resize:
      # Expand dimensions to make it 3D
      imagens_preprocessadas = tf.expand_dims(imagens_preprocessadas, axis=2)

      # Resize the image
      imagens_preprocessadas = tf.image.resize(imagens_preprocessadas, [W, L])

      # Reshape the image to (28, 28, 1) to make it compatible with Conv2D input shape
      imagens_preprocessadas = tf.reshape(imagens_preprocessadas, [W, L, 1])

      # Remove the extra dimension added by tf.expand_dims
      imagens_preprocessadas = tf.squeeze(imagens_preprocessadas, axis=-1)  # Squeeze the last dimension

    return imagens_preprocessadas

In [ ]:
# Caminhos para os diretórios
caminho_csv = "/content/drive/MyDrive/Colab LFUI/Colab Notebooks/data set/data_18_12_23/CSV"
caminho_imagens = "/content/drive/MyDrive/Colab LFUI/Colab Notebooks/Alex_dataset"

# Listar arquivos CSV
arquivos_csv = [arq for arq in os.listdir(caminho_csv) if arq.endswith('.csv')]
tamanho_do_set = len(arquivos_csv)
print(f'O tamanho do arquivos_csv é: {tamanho_do_set}', end='\n\n')

# Listar arquivos PNG
arquivos_png = [arq for arq in os.listdir(caminho_imagens) if arq.endswith('.png')]
tamanho_do_set = len(arquivos_png)
print(f'O tamanho do arquivos_png é: {tamanho_do_set}', end='\n\n')

# Extrair timestamps dos arquivos CSV
timestamps_csv = set([arquivo_csv.split('_')[3].split('.')[0] for arquivo_csv in arquivos_csv])
tamanho_do_set = len(timestamps_csv)
print(f'O tamanho do timestamps_csv é: {tamanho_do_set}', end='\n\n')

# Extrair timestamps dos arquivos PNG
timestamps_png = set([arquivo_png.split('_')[3].split('.')[0] for arquivo_png in arquivos_png])
tamanho_do_set = len(timestamps_png)
print(f'O tamanho do timestamps_png é: {tamanho_do_set}', end='\n\n')

# Exibir timestamps únicos de cada lista
print("Timestamps únicos CSV:")
print(timestamps_csv)

print("\nTimestamps únicos PNG:")
print(timestamps_png)

# Verificar se há correspondência de timestamps entre os arquivos CSV e PNG
correspondencia_timestamp = timestamps_csv.intersection(timestamps_png)
print(f"\nCorrespondência de timestamps:")
print(correspondencia_timestamp, end='\n\n')
tamanho_do_set = len(correspondencia_timestamp)
print(f'O tamanho do correspondencia_timestamp é: {tamanho_do_set}')

O tamanho do arquivos_csv é: 506

O tamanho do arquivos_png é: 505

O tamanho do timestamps_csv é: 506

O tamanho do timestamps_png é: 505

Timestamps únicos CSV:
{'153238', '152315', '154130', '153259', '154411', '153716', '153803', '153350', '153308', '152505', '150524', '152212', '152051', '153214', '152556', '154206', '154426', '154039', '152637', '154054', '153356', '153425', '152136', '153311', '152937', '152954', '153754', '153848', '153827', '153057', '152602', '153206', '152015', '154330', '152432', '152925', '153247', '152124', '152704', '152734', '154000', '152227', '154200', '152913', '153727', '152426', '152928', '152438', '152520', '154003', '152224', '154227', '153456', '152100', '154136', '152200', '153241', '153139', '154420', '154433', '154148', '152511', '153628', '152934', '153842', '152707', '152523', '153438', '152822', '153305', '152417', '153658', '153408', '154342', '154400', '152948', '153707', '153317', '153353', '153405', '154127', '153402', '154333', '15230

In [ ]:
# Criar um dicionário para armazenar os elementos correspondentes
dados_por_timestamp = {}
scaler = StandardScaler()

# Iterar sobre os arquivos CSV
for arquivo_csv in arquivos_csv:
    timestamp = arquivo_csv.split('_')[3].split('.')[0]
    caminho_completo_csv = os.path.join(caminho_csv, arquivo_csv)

    # Ler o conteúdo do arquivo CSV e extrair os 3 valores relevantes
    with open(caminho_completo_csv, 'r') as f_csv:
        conteudo_csv = f_csv.read()
        valores_zernike = np.array([round(float(valor), 1) for valor in conteudo_csv.split(',')[3:6]])

    # Verificar se o timestamp já existe no dicionário
    if timestamp not in dados_por_timestamp:
        dados_por_timestamp[timestamp] = {'zernike': valores_zernike}
    else:
        dados_por_timestamp[timestamp]['zernike'] = valores_zernike

# Iterar sobre os arquivos PNG
for arquivo_png in arquivos_png:
    timestamp = arquivo_png.split('_')[3].split('.')[0]
    caminho_completo_png = os.path.join(caminho_imagens, arquivo_png)

    # Ler a imagem PNG e realizar qualquer pré-processamento necessário
    imagem = cv2.imread(caminho_completo_png, cv2.IMREAD_GRAYSCALE)  # Carregar imagem em escala de cinza (se necessário)

    imagens_preprocessadas = preprocess_image(imagem)

    # Verificar se o timestamp já existe no dicionário
    if timestamp not in dados_por_timestamp:
        dados_por_timestamp[timestamp] = {'imagem': imagens_preprocessadas}
    else:
        dados_por_timestamp[timestamp]['imagem'] = imagens_preprocessadas

# Agora, 'dados_por_timestamp' contém os elementos correspondentes agrupados pelo timestamp
# Estruture esses dados conforme necessário para treinar sua rede neural

# Exemplo de como acessar os dados para um timestamp específico
timestamp_exemplo = '154453'
dados_exemplo = dados_por_timestamp[timestamp_exemplo]
print(f'Dados para o timestamp {timestamp_exemplo}: {dados_exemplo}')

Dados para o timestamp 154453: {'zernike': array([-0.1,  0.7,  0.1]), 'imagem': <tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.4533   ,  2.31     ,  2.3135004, ...,  1.2465017,  1.2001007,
         1.769989 ],
       [ 2.6998   ,  2.0403001,  1.1499996, ...,  1.1304947,  1.1300001,
         1.1000986],
       [ 2.       ,  1.6895001,  1.0674998, ...,  1.1499939,  1.5500002,
         1.4234941],
       ...,
       [41.395084 , 27.13695  , 25.114985 , ...,  4.549988 ,  5.031504 ,
         3.230011 ],
       [38.650143 , 29.540941 , 20.698978 , ...,  4.130496 ,  4.       ,
         3.0298991],
       [33.       , 25.8107   , 18.625021 , ...,  4.149994 ,  3.7102108,
         3.       ]], dtype=float32)>}


In [ ]:
# Lista de timestamps no conjunto de dados
timestamps = list(dados_por_timestamp.keys())
rotulos = np.array([dados_por_timestamp[timestamp]['zernike'] for timestamp in timestamps])
imagens = np.array([dados_por_timestamp[timestamp]['imagem'] for timestamp in timestamps])

# Divisão dos timestamps em treinamento, validação e teste
timestamps_treino, timestamps_teste = train_test_split(timestamps, test_size=0.2, random_state=42)
timestamps_validacao, timestamps_teste = train_test_split(timestamps_teste, test_size=0.5, random_state=42)

# Função para extrair os dados correspondentes a um conjunto de timestamps
def obter_dados_por_timestamps(conjunto_timestamps):
    return {timestamp: dados_por_timestamp[timestamp] for timestamp in conjunto_timestamps}

# Obter dados para conjuntos de treinamento, validação e teste
dados_treino = obter_dados_por_timestamps(timestamps_treino)
dados_validacao = obter_dados_por_timestamps(timestamps_validacao)
dados_teste = obter_dados_por_timestamps(timestamps_teste)

KeyError: 'imagem'

In [ ]:
print("Conjunto de Treinamento:")
print("Número de timestamps:", len(timestamps_treino))
print("Exemplo de timestamp:", timestamps_treino[0])
print("Exemplo de dados associados:", dados_treino[timestamps_treino[0]])

print("\nConjunto de Validação:")
print("Número de timestamps:", len(timestamps_validacao))
print("Exemplo de timestamp:", timestamps_validacao[0])
print("Exemplo de dados associados:", dados_validacao[timestamps_validacao[0]])

print("\nConjunto de Teste:")
print("Número de timestamps:", len(timestamps_teste))
print("Exemplo de timestamp:", timestamps_teste[0])
print("Exemplo de dados associados:", dados_teste[timestamps_teste[0]])

In [ ]:
# Função para criar um tensor a partir dos dados
def criar_tensor(dados_por_timestamp):
    timestamps = list(dados_por_timestamp.keys())
    rotulos = np.array([dados_por_timestamp[timestamp]['zernike'] for timestamp in timestamps])
    imagens = np.array([dados_por_timestamp[timestamp]['imagem'] for timestamp in timestamps])

    return tf.data.Dataset.from_tensor_slices((imagens, rotulos))

# Criar conjuntos de treinamento, validação e teste
conjunto_treino = criar_tensor(dados_treino)
conjunto_validacao = criar_tensor(dados_validacao)
conjunto_teste = criar_tensor(dados_teste)

# Embaralhar e agrupar os conjuntos
BATCH_SIZE = 32
conjunto_treino = conjunto_treino.shuffle(len(dados_treino)).batch(BATCH_SIZE)
conjunto_validacao = conjunto_validacao.batch(BATCH_SIZE)
conjunto_teste = conjunto_teste.batch(BATCH_SIZE)

In [ ]:
# Visualizar alguns exemplos do conjunto de treinamento
for imagem, rotulo in conjunto_treino.take(5):
    print(f"Rotulo: {rotulo.numpy()}")
    print(f"Shape da Imagem: {imagem.shape}")
    print("Valores da Imagem:")
    print(imagem.numpy())
    print("\n" + "="*30 + "\n")

In [ ]:
# Defina a arquitetura da sua rede neural
def criar_modelo():
    modelo = models.Sequential()

    # Adicione camadas convolucionais e de pooling
    modelo.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu'))
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Adicione camadas densas
    modelo.add(layers.Flatten())
    modelo.add(layers.Dense(64, activation='relu'))

    # Adicione a camada de saída com 3 neurônios (um para cada coeficiente Zernike)
    modelo.add(layers.Dense(3))

    return modelo

# Crie uma instância do modelo
modelo = criar_modelo()

# Compile o modelo
modelo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])

# Exiba um resumo da arquitetura do modelo
modelo.summary()

In [ ]:
tf.keras.utils.plot_model(
 modelo,
 to_file='model_plot_CNN.png',
 show_shapes=True,
 show_layer_names=True,
 rankdir='TB',
 expand_nested=True,
 dpi=96,
)

In [ ]:
# Treinamento do modelo
historico_treino = modelo.fit(
    conjunto_treino,
    epochs=20,  # Ajuste o número de épocas conforme necessário
    validation_data=conjunto_validacao
)

# Avaliação do modelo
resultado_avaliacao = modelo.evaluate(conjunto_teste)
print(f'Avaliação do modelo nos dados de teste: {resultado_avaliacao}')

In [ ]:
# Extrair os resultados do histórico de treinamento
loss = historico_treino.history['loss']
mae = historico_treino.history['mae']
val_loss = historico_treino.history['val_loss']
val_mae = historico_treino.history['val_mae']

# Número de épocas
epocas = range(1, len(loss) + 1)

# Plotar a perda
plt.plot(epocas, loss, 'bo', label='Treinamento loss')
plt.plot(epocas, val_loss, 'b', label='Validação loss')
plt.title('Perda de Treinamento e Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()

# Plotar o MAE
plt.plot(epocas, mae, 'bo', label='Treinamento MAE')
plt.plot(epocas, val_mae, 'b', label='Validação MAE')
plt.title('Erro Absoluto Médio de Treinamento e Validação')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# Extrair a acurácia do histórico de treinamento
accuracy = historico_treino.history['accuracy']
val_accuracy = historico_treino.history['val_accuracy']

# Avaliação do modelo nos dados de teste
resultado_teste = modelo.evaluate(conjunto_teste)

# Extrair a acurácia do resultado de teste
accuracy_teste = resultado_teste[2]  # A posição 2 corresponde à acurácia, ajuste conforme necessário

# Número de épocas
epocas = range(1, len(accuracy) + 1)

# Plotar a acurácia
plt.plot(epocas, accuracy, 'bo', label='Treinamento accuracy')
plt.plot(epocas, val_accuracy, 'b', label='Validação accuracy')
plt.axhline(y=accuracy_teste, color='r', linestyle='--', label='Teste accuracy')  # Adicionar linha para a acurácia do teste
plt.title('Acurácia de Treinamento, Validação e Teste')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.show()